In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder, OneHotEncoder
import tensorflow as tf
from sklearn.compose import ColumnTransformer, make_column_selector
import os

d:\meridianthe4\PML\tf\Lib\site-packages\keras\src\export\tf2onnx_lib.py:8: FutureWarning: In the future `np.object` will be defined as the corresponding NumPy scalar.
  if not hasattr(np, "object"):


In [2]:
os.chdir("D:\\meridianthe4\\PML\\Cases\\Glass_Identification")

In [3]:
glass = pd.read_csv('Glass.csv')

In [4]:
ohe = OneHotEncoder(drop='first', sparse_output=False).set_output(transform="pandas")
col_transformer = ColumnTransformer([("OHE", ohe, make_column_selector(dtype_include=object))], 
                                    remainder='passthrough', 
                                    verbose_feature_names_out=False)
col_transformer = col_transformer.set_output(transform="pandas")

In [5]:
X, y = glass.drop(columns=['Type']), glass['Type']
le = LabelEncoder()
y = le.fit_transform(y)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=25)
X_train = col_transformer.fit_transform(X_train)
X_test = col_transformer.transform(X_test)
scl_X = StandardScaler()
X_train_scaled = scl_X.fit_transform(X_train)
X_test_scaled = scl_X.transform(X_test)

In [6]:
reg_l1 = tf.keras.regularizers.L1(0.1)
initializer = tf.keras.initializers.GlorotUniform(seed=25)
tf.random.set_seed(25)
model = tf.keras.Sequential([tf.keras.Input(shape=(X_train_scaled.shape[1],)),
                             tf.keras.layers.Dense(14, activation='relu', kernel_initializer=initializer, kernel_regularizer=reg_l1),
                                tf.keras.layers.Dense(10, activation='relu', kernel_initializer=initializer, kernel_regularizer=reg_l1),
                                tf.keras.layers.Dense(7, activation='relu', kernel_initializer=initializer, kernel_regularizer=reg_l1),
                                tf.keras.layers.Dense(6, activation='softmax', kernel_initializer=initializer)])
model.compile(optimizer=tf.keras.optimizers.RMSprop(learning_rate=0.001), loss=tf.keras.losses.SparseCategoricalCrossentropy(), metrics=['acc'])
history = model.fit(X_train_scaled, y_train, validation_data=(X_test_scaled, y_test), epochs=500)

Epoch 1/500
5/5 ━━━━━━━━━━━━━━━━━━━━ 1s 46ms/step - acc: 0.0872 - loss: 9.9922 - val_acc: 0.1077 - val_loss: 9.7866
Epoch 2/500
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - acc: 0.1141 - loss: 9.7014 - val_acc: 0.1077 - val_loss: 9.5609
Epoch 3/500
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - acc: 0.1342 - loss: 9.4934 - val_acc: 0.1231 - val_loss: 9.3687
Epoch 4/500
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - acc: 0.1745 - loss: 9.3101 - val_acc: 0.1846 - val_loss: 9.1906
Epoch 5/500
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - acc: 0.2215 - loss: 9.1382 - val_acc: 0.2000 - val_loss: 9.0210
Epoch 6/500
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - acc: 0.2349 - loss: 8.9742 - val_acc: 0.2923 - val_loss: 8.8573
Epoch 7/500
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - acc: 0.2416 - loss: 8.8145 - val_acc: 0.3385 - val_loss: 8.6965
Epoch 8/500
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - acc: 0.2685 - loss: 8.6563 - val_acc: 0.3692 - val_loss: 8.5375
Epoch 9/500
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step - acc: 0.2953 - loss: 

In [7]:
from sklearn.metrics import accuracy_score, log_loss
y_pred_proba = model.predict(X_test_scaled)
print(log_loss(y_test, y_pred_proba))
y_pred = y_pred_proba.argmax(axis=1)
print(accuracy_score(y_test, y_pred))

3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1.4433872029400518
0.38461538461538464


In [10]:
from tensorflow.keras.callbacks import EarlyStopping